In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory

from pyomo_models.base import BaseModel
from pyomo_models.multi_uavs import MultiUavModel
from util.scenario import Scenario
from simulate.simulate import Simulation
from util import constants

# Base model

In [ ]:
sc = Scenario.from_file("scenarios/single_drone.yml")

_, ax = plt.subplots(figsize=(7,7), dpi=80)
sc.plot(ax=ax)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
# PARAMETERS
B_min = 0
B_max = 1
B_start = [1]
v = [1]
r_charge = np.array([0.1])
r_deplete = np.array([0.25])

parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
model = BaseModel(sc, parameters)

In [ ]:
# solution = SolverFactory('cbc').solve(model, tee=True) 
solution = SolverFactory('cbc').solve(model) 
time_taken = solution['solver'][0]['time']
print(f"Took {time_taken:.3f}s to find charging schedule")

In [ ]:
print(f"The optimal execution time: {model.execution_time():.2f}s")
charging_time = sum(np.array(model.C[:,:]()))
print(f"Of this, the drone(s) spent {charging_time:.2f}s charging")
print(f"Which means that the drone(s) spent {model.execution_time() - charging_time:.2f}s flying")

In [ ]:
axes = plt.subplots(1,2, figsize=(15,30), dpi=80) 
ax1, ax2= axes[1]

# problem
sc.plot(ax=ax1, draw_distances=False)
ax1.set_aspect('equal', adjustable='box')
ax1.axis('off')

model.plot(ax=ax2)
# plot_second_solved(model, positions_S, positions_w, ax=ax2)
ax2.set_aspect('equal', adjustable='box')
ax2.axis('off')

plt.show()

# Multi-UAV model

In [ ]:
# PARAMETERS
B_min = 0
B_max = 1
B_start = [1, 1]
v = [1, 1]
r_charge = np.array([0.1, 0.1])
r_deplete = np.array([0.24, 0.24])

sc = Scenario.from_file("scenarios/two_drones.yml")

In [ ]:
_, ax = plt.subplots(figsize=(7,7), dpi=80)
sc.plot(ax=ax, draw_distances=False)
ax.set_aspect('equal', adjustable='box')
ax.axis('off')
plt.show()

In [ ]:
parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
parameters['epsilon'] = 0
model = MultiUavModel(sc, parameters)
model.initialize_variables(seed=1)

In [ ]:
# solution = SolverFactory('cbc').solve(model, tee=True) 
solver = SolverFactory("mindtpy")
solution = solver.solve(model, mip_solver='gurobi', nlp_solver='ipopt')

In [ ]:
t = solution['Solver'][0]['Best solution found time']
print(f"Took {t:.2f}s to solve")

In [ ]:
axes = plt.subplots(1,2, figsize=(15,30), dpi=80) 
ax1, ax2= axes[1]

# problem
sc.plot(ax=ax1, draw_distances=False)
ax1.set_aspect('equal', adjustable='box')
ax1.axis('off')

model.plot(ax=ax2)
ax2.set_aspect('equal', adjustable='box')
ax2.axis('off')

plt.savefig("out/two_drones.pdf", bbox_inches="tight")
plt.show()

In [ ]:
_, axes = plt.subplots(model.N_d, 1, figsize=(5,model.N_d), dpi=80, sharex=True, sharey=True)

for d in model.d:
    Simulation.from_base_model(model, d).plot_charge(ax=axes[d], color=constants.W_COLORS[d])
    if d != model.N_d - 1:
        axes[d].set_xlabel("")
    axes[d].set_xticks([])
axes[model.N_d-1].set_xlabel("Time")

plt.savefig("out/two_drones_battery.pdf", bbox_inches="tight")
plt.show()